<a href="https://colab.research.google.com/github/SAnderwald/Certified-Solution-Architect-Associate-Guide/blob/master/Copy_of_Untitled5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import cv2
import pickle
import numpy as np
import os
import tensorflow as tf
from google.colab import drive

In [2]:
drive.mount('/content/drive')

Mounted at /content/drive


In [7]:
import os
import cv2
import numpy as np
import pickle
import tensorflow as tf

# Define function to overlay annotations on video frames
def overlay_annotations(video_path, annotations_path, output_file, batch_size=10):
    print("Start processing...")

    # Load video
    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        print("Error: Could not open video file")
        return

    # Get video properties
    fps = cap.get(cv2.CAP_PROP_FPS)
    frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

    # Define codec and create VideoWriter object
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    out = cv2.VideoWriter(output_file, fourcc, fps, (frame_width, frame_height))

    # Load annotations
    with open(annotations_path, 'rb') as f:
        annotations = pickle.load(f)

    # Process video in batches
    print("Processing video frames in batches...")
    frame_number = 0
    while frame_number < frame_count:
        frames = []
        ret = True
        for _ in range(batch_size):
            if frame_number >= frame_count:
                break
            ret, frame = cap.read()
            if not ret:
                break
            frames.append(frame)
            frame_number += 1

        # Process each frame in the batch
        for i, frame in enumerate(frames):
            current_frame_number = frame_number - len(frames) + i
            if current_frame_number in annotations:
                for person_id, (bbox, keypoints) in annotations[current_frame_number].items():
                    x, y, w, h = map(int, bbox)
                    cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)
            out.write(frame)

        print(f"Processed frame batch {frame_number - len(frames) + 1} to {frame_number}")

    # Release resources
    cap.release()
    out.release()

    print("Processing completed successfully")

# Define function to overlay annotations and labels on video frames
def overlay_annotations_and_labels(video_path, annotations_path, anomaly_labels_path, output_file, batch_size=10):
    print("Start processing...")

    # Load video
    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        print("Error: Could not open video file")
        return

    # Get video properties
    fps = cap.get(cv2.CAP_PROP_FPS)
    frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

    # Define codec and create VideoWriter object
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    out = cv2.VideoWriter(output_file, fourcc, fps, (frame_width, frame_height))

    # Load annotations
    with open(annotations_path, 'rb') as f:
        annotations = pickle.load(f)

    # Load anomaly labels
    anomaly_labels = np.load(anomaly_labels_path)

    # Process video in batches
    print("Processing video frames in batches...")
    frame_number = 0
    while frame_number < frame_count:
        frames = []
        ret = True
        for _ in range(batch_size):
            if frame_number >= frame_count:
                break
            ret, frame = cap.read()
            if not ret:
                break
            frames.append(frame)
            frame_number += 1

        # Process each frame in the batch
        for i, frame in enumerate(frames):
            current_frame_number = frame_number - len(frames) + i
            if current_frame_number in annotations:
                for person_id, (bbox, keypoints) in annotations[current_frame_number].items():
                    x, y, w, h = map(int, bbox)
                    label = 1 if anomaly_labels[current_frame_number] == 1 else 0
                    color = (0, 0, 255) if label == 1 else (0, 255, 0)
                    cv2.rectangle(frame, (x, y), (x + w, y + h), color, 2)
                    cv2.putText(frame, f'Label: {label}', (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)
            out.write(frame)

    # Release resources
    cap.release()
    out.release()

    print("Processing completed successfully")

# Define function to process all videos in a directory
def process_video_directory(video_dir, annotations_dir, labels_dir, output_dir, batch_size=10):
    # Get list of video files
    video_files = os.listdir(video_dir)

    # Process each video file
    for video_file in video_files:
        video_name = os.path.splitext(video_file)[0]
        video_path = os.path.join(video_dir, video_file)
        annotations_path = os.path.join(annotations_dir, video_name + '.pkl')
        labels_path = os.path.join(labels_dir, video_name + '.npy')
        output_file = os.path.join(output_dir, video_name + '_annotated_with_labels.mp4')

        if os.path.exists(video_path) and os.path.exists(annotations_path) and os.path.exists(labels_path):
            overlay_annotations_and_labels(video_path, annotations_path, labels_path, output_file, batch_size)
        else:
            print(f"Missing files for video: {video_file}")




# Define file paths
video_dir = '/content/drive/MyDrive/Dissertation_Project/data/raw/video'
annotations_dir = '/content/drive/MyDrive/Dissertation_Project/data/raw/annotations'
labels_dir = '/content/drive/MyDrive/Dissertation_Project/data/raw/anomaly_labels'
output_dir = '/content/drive/MyDrive/Dissertation_Project/data/processed/Overlays'

# Overlay annotations and labels on all videos in the directory
process_video_directory(video_dir, annotations_dir, labels_dir, output_dir)




Start processing...
Processing video frames in batches...
Processing completed successfully
Start processing...
Processing video frames in batches...
Processing completed successfully
Start processing...
Processing video frames in batches...
Processing completed successfully
Start processing...
Processing video frames in batches...
Processing completed successfully
Start processing...
Processing video frames in batches...
Processing completed successfully
Start processing...
Processing video frames in batches...
Processing completed successfully
Start processing...
Processing video frames in batches...
Processing completed successfully
Start processing...
Processing video frames in batches...
Processing completed successfully
Start processing...
Processing video frames in batches...
Processing completed successfully
Start processing...
Processing video frames in batches...
Processing completed successfully
Start processing...
Processing video frames in batches...
Processing completed s

In [ ]:
from google.colab import drive
import os

# Mount Google Drive
drive.mount('/content/drive')

# Check if the drive is mounted
if os.path.isdir('/content/drive/MyDrive'):
    print("Drive mounted successfully.")
    # Check specific directories
    directories = [
        '/content/drive/MyDrive/Dissertation_Project/data/raw/video',
        '/content/drive/MyDrive/Dissertation_Project/data/raw/annotations',
        '/content/drive/MyDrive/Dissertation_Project/data/raw/anomaly_labels',
        '/content/drive/MyDrive/Dissertation_Project/data/processed'
    ]

    for directory in directories:
        if os.path.isdir(directory):
            print(f"Directory exists: {directory}")
        else:
            print(f"Directory does not exist: {directory}")
else:
    print("Error: Drive not mounted.")


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Drive mounted successfully.
Directory exists: /content/drive/MyDrive/Dissertation_Project/data/raw/video
Directory exists: /content/drive/MyDrive/Dissertation_Project/data/raw/annotations
Directory exists: /content/drive/MyDrive/Dissertation_Project/data/raw/anomaly_labels
Directory exists: /content/drive/MyDrive/Dissertation_Project/data/processed
